# FTC Do Not Call Registry Analysis

by [Nate Rattner](http://naterattner.com/)  
This code analyzes datasets published by the Federal Trade Commission, which contain information about phone numbers on the FTC's Do Not Call Registry as well as complaints consumers submit to the FTC about unwanted phone calls. This data was used for multiple charts and a story published on Medium, both of which can be found [here](https://medium.com/@naterattner/the-rise-of-robocalls-in-six-charts-37787ac8402e). The visuals used in that post are also available in a story made with Flourish, which can be seen [here]( https://public.flourish.studio/story/39208/).

In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

## Per Capita Do Not Call Registrations by Fiscal Year

In [2]:
#Load FTC Do Not Call registration data that includes registrations by FY (https://www.ftc.gov/reports/national-do-not-call-registry-data-book-fiscal-year-2018)
regs_by_fy = pd.read_csv('do_not_call_registry_data_book_data_files_fy2018_0/2018_DNC_Registrations_OrganizationsAccess_5yr.csv', header=1)

#Select the rows and columns that have the relevant data
regs_by_fy = regs_by_fy[0:5]
regs_by_fy = regs_by_fy[['FY', 'Registrations']]
regs_by_fy

,FY,Registrations
0,2014,"217,855,796"
1,2015,"222,841,544"
2,2016,"226,001,288"
3,2017,"229,816,164"
4,2018,"235,302,818"


In [3]:
#Change Registrations column from object to int. Have to remove commas first
regs_by_fy['Registrations'] = regs_by_fy['Registrations'].str.replace(',', '')

regs_by_fy['Registrations'] = regs_by_fy.Registrations.astype(int)

regs_by_fy

,FY,Registrations
0,2014,217855796
1,2015,222841544
2,2016,226001288
3,2017,229816164
4,2018,235302818


In [4]:
#Create list of 2014-2018 population data from U.S. Census (source: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html)
pop_data_14_18 = [318386421, 320742673, 323071342, 325147121, 327167434]

#Add this list to the registrations by FY dataframe
regs_by_fy['annual-pop'] = pop_data_14_18

#Calculate registrations per capita
regs_by_fy['registrationspercapita'] = (regs_by_fy['Registrations']/regs_by_fy['annual-pop'])*100000

#Change Registrations per capita to int
regs_by_fy['registrationspercapita'] = regs_by_fy.registrationspercapita.astype(int)

regs_by_fy

#Export this data
#regs_by_fy.to_csv('regs_by_fy_per_capita.csv', index = None, header=True)

,FY,Registrations,annual-pop,registrationspercapita
0,2014,217855796,318386421,68424
1,2015,222841544,320742673,69476
2,2016,226001288,323071342,69953
3,2017,229816164,325147121,70680
4,2018,235302818,327167434,71921


## Do Not Call Complaints by Fiscal Year and Call Type

In [5]:
#Load FTC Do Not Call registration data that includes complaints by call type and FY (https://www.ftc.gov/reports/national-do-not-call-registry-data-book-fiscal-year-2018)

comps = pd.read_csv('do_not_call_registry_data_book_data_files_fy2018_0/2018_DNC_Complaints_by_CallType_5yr.csv', header=1)

#Select the rows and columns that have the FY data
comps_by_fy = comps.loc[3].to_frame().reset_index()
comps_by_fy = comps_by_fy[1:].reset_index()
comps_by_fy.columns = ['null','FY','Complaints']
comps_by_fy = comps_by_fy[['FY','Complaints']]
comps_by_fy

#Export this data
#comps_by_fy.to_csv('comps_by_fy.csv', index = None, header=True)

,FY,Complaints
0,2014,"3,241,050"
1,2015,"3,578,695"
2,2016,"5,340,207"
3,2017,"7,157,337"
4,2018,"5,780,172"


In [6]:
#Select the rows and columns that have the call type data
comps_by_type = comps[0:3]
comps_by_type

#Export this data
#comps_by_type.to_csv('comps_by_type.csv', index = None, header=True)

,CallType,2014,2015,2016,2017,2018
0,Live Caller,"1,448,363","1,388,455","1,854,695","2,563,055","1,894,327"
1,Robocall,"1,734,586","2,125,968","3,401,610","4,501,960","3,790,614"
2,Call Type Not Reported,"58,101","64,272","83,902","92,322","95,231"


## Do Not Call Registrations by State, FY 18

In [7]:
#Load FTC National Do Not Call registration data (https://www.ftc.gov/reports/national-do-not-call-registry-data-book-fiscal-year-2018)
regs_by_state = pd.read_csv('do_not_call_registry_data_book_data_files_fy2018_0/2018_DNC_State_Registrations_5y.csv', header=1)
regs_by_state.head()

,State,FY,Registrations
0,Alaska,2014,348309
1,Alabama,2014,3202106
2,Arkansas,2014,1906859
3,Arizona,2014,4378074
4,California,2014,24700367


In [8]:
#Keep only FY 18 data
regs_by_state_18 = regs_by_state.loc[regs_by_state['FY'] == 2018].reset_index()[['State','Registrations']]
regs_by_state_18.head()

,State,Registrations
0,Alaska,375719
1,Alabama,3487166
2,Arkansas,2087259
3,Arizona,4710641
4,California,26447425


In [9]:
#Load population data
state_pop = pd.read_csv('state-pop-data.csv')

#Keep only the columns needed
state_pop_18 = state_pop[['NAME','STATE-ABBREV','POPESTIMATE2018']]
state_pop_18.head()

,NAME,STATE-ABBREV,POPESTIMATE2018
0,United States,NaN,327167434
1,Northeast Region,NaN,56111079
2,Midwest Region,NaN,68308744
3,South Region,NaN,124753948
4,West Region,NaN,77993663


In [10]:
#Join population data to DNC registration data on state
regs_by_state_18_joined = regs_by_state_18.join(state_pop_18.set_index('NAME'), on='State')
regs_by_state_18_joined.head()

,State,Registrations,STATE-ABBREV,POPESTIMATE2018
0,Alaska,375719,AK,737438
1,Alabama,3487166,AL,4887871
2,Arkansas,2087259,AR,3013825
3,Arizona,4710641,AZ,7171646
4,California,26447425,CA,39557045


In [11]:
#Add columns calculating Do Not Call registrations per capita (registrations per 100,000 people)
regs_by_state_18_joined['registrations-per-capita-18'] = (regs_by_state_18_joined['Registrations']/regs_by_state_18_joined['POPESTIMATE2018'])*100000
regs_per_capita_18 = regs_by_state_18_joined.sort_values('registrations-per-capita-18', ascending=False)

#Round the registrations-per-capita column (change from float to integer)
regs_per_capita_18[('registrations-per-capita-18')] = regs_per_capita_18[('registrations-per-capita-18')].astype(int)

#Reset index
regs_per_capita_18 = regs_per_capita_18.reset_index()[['State','Registrations','STATE-ABBREV','POPESTIMATE2018','registrations-per-capita-18']]

regs_per_capita_18.head()

#Export this data
#regs_per_capita_18.to_csv('regs_per_capita_18.csv', index = None, header=True)

,State,Registrations,STATE-ABBREV,POPESTIMATE2018,registrations-per-capita-18
0,New Hampshire,1245599,NH,1356458,91827
1,District of Columbia,638132,DC,702455,90843
2,Connecticut,3234535,CT,3572665,90535
3,Massachusetts,5893707,MA,6902149,85389
4,Maine,1121496,ME,1338404,83793


In [12]:
#Create choropleth map to check variation in state registrations

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]


data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = regs_per_capita_18['STATE-ABBREV'],
    z = regs_per_capita_18['registrations-per-capita-18'].astype(float),
    locationmode = 'USA-states',
    #text = regs_per_capita_18['State'] + regs_per_capita_18['Registrations'] + '<br>Total Registrations:' ,
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 2
        )),
    colorbar = go.choropleth.ColorBar(
        title = "Registrations Per Capita")
)]


layout = go.Layout(
    title = go.layout.Title(
        text = 'PLACEHOLDER TITLE'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = False,
        lakecolor = 'rgb(255, 255, 255)'),
)


fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'd3-cloropleth-map')

## Do Not Call Registrations by State, Percent Change FY 14 to FY 18

In [13]:
#Create dataframe with only FY 14 data
regs_by_state_14 = regs_by_state.loc[regs_by_state['FY'] == 2014].reset_index()[['State','Registrations']]
regs_by_state_14.head()

,State,Registrations
0,Alaska,348309
1,Alabama,3202106
2,Arkansas,1906859
3,Arizona,4378074
4,California,24700367


In [14]:
#Create dataframe with 2014 population data
state_pop_14 = state_pop[['NAME','STATE-ABBREV','POPESTIMATE2014']]
state_pop_14.head()

,NAME,STATE-ABBREV,POPESTIMATE2014
0,United States,NaN,318386421
1,Northeast Region,NaN,56015864
2,Midwest Region,NaN,67752238
3,South Region,NaN,119657737
4,West Region,NaN,74960582


In [15]:
#Join population data to Do Not Call registration data on state
regs_by_state_14_joined = regs_by_state_14.join(state_pop_14.set_index('NAME'), on='State')
regs_by_state_14_joined.head()

,State,Registrations,STATE-ABBREV,POPESTIMATE2014
0,Alaska,348309,AK,736307
1,Alabama,3202106,AL,4842481
2,Arkansas,1906859,AR,2967726
3,Arizona,4378074,AZ,6733840
4,California,24700367,CA,38625139


In [16]:
#Add columns calculating Do Not Call registrations per capita (registrations per 100,000 people)
regs_by_state_14_joined['registrations-per-capita-14'] = (regs_by_state_14_joined['Registrations']/regs_by_state_14_joined['POPESTIMATE2014'])*100000
regs_per_capita_14 = regs_by_state_14_joined.sort_values('registrations-per-capita-14', ascending=False)

#Round the per-capita column (change from float to integer)
regs_per_capita_14[('registrations-per-capita-14')] = regs_per_capita_14[('registrations-per-capita-14')].astype(int)

#Reset index
regs_per_capita_14 = regs_per_capita_14.reset_index()[['State','Registrations','STATE-ABBREV','POPESTIMATE2014','registrations-per-capita-14']]

regs_per_capita_14.head()

,State,Registrations,STATE-ABBREV,POPESTIMATE2014,registrations-per-capita-14
0,District of Columbia,575799,DC,662513,86911
1,New Hampshire,1155347,NH,1333223,86658
2,Connecticut,3020552,CT,3594783,84025
3,Massachusetts,5531792,MA,6763652,81787
4,Colorado,4261903,CO,5351218,79643


In [17]:
#Join the per capita dataframes
regs_per_capita_14_18 = regs_per_capita_14.merge(regs_per_capita_18, on=['State', 'STATE-ABBREV'])

#Keep only the relevant columns
regs_per_capita_14_18 = regs_per_capita_14_18[['State','STATE-ABBREV','registrations-per-capita-14','registrations-per-capita-18']]

#Calculate percent change from 2014 to 2018
regs_per_capita_14_18[('per-capita-change')] = ((regs_per_capita_14_18['registrations-per-capita-18'] - regs_per_capita_14_18['registrations-per-capita-14'])/regs_per_capita_14_18['registrations-per-capita-14']) * 100
regs_per_capita_14_18.sort_values('per-capita-change', ascending=False).head()


#Export this data
#regs_per_capita_14_18.to_csv('regs_per_capita_14_18.csv', index = None, header=True)

,State,STATE-ABBREV,registrations-per-capita-14,registrations-per-capita-18,per-capita-change
51,Puerto Rico,NaN,10784,12616,16.988131
42,West Virginia,WV,62962,70955,12.694959
45,Indiana,IN,58066,64193,10.551786
49,Mississippi,MS,51140,56413,10.310911
17,Wyoming,WY,74479,81552,9.496637


## Do Not Call Complaints by Topic, FY 18

In [18]:
comps_by_topic = pd.read_csv('do_not_call_registry_data_book_data_files_fy2018_0/2018_DNC_Complaints_by_Topic_CallType_Month.csv', header=1)

#Select only the rows needed
comps_by_topic = comps_by_topic[0:11]

#Select only the columns needed
comps_by_topic = comps_by_topic[['Topic','Live Caller','Robocall']]

comps_by_topic

#Export this data
#comps_by_topic.to_csv('comps_by_topic.csv', index = None, header=True)


,Topic,Live Caller,Robocall
0,Reducing debt,"87,632","641,734"
1,Medical & prescriptions,"101,020","340,439"
2,Imposters,"77,743","315,245"
3,Vacation & timeshares,"37,044","120,212"
4,Computer & technical support,"53,165","85,876"
5,Warranties & protection plans,"45,544","73,900"
6,"Energy, solar, & utilities","44,900","43,938"
7,Home improvement & cleaning,"37,096","44,991"
8,Home security & alarms,"11,589","41,429"
9,"Lotteries, prizes & sweepstakes","13,937","36,200"


## Do Not Call Complaints by Month, FY 18

In [19]:
comps_by_month = pd.read_csv('do_not_call_registry_data_book_data_files_fy2018_0/2018_DNC_Complaints_by_Topic_CallType_Month.csv', header=22)

#Select only the rows needed
comps_by_month = comps_by_month[0:12]

comps_by_month

#Export this data
#comps_by_month.to_csv('comps_by_month.csv', index = None, header=True)

,Month,Live Caller,Robocall,Call Type Not Reported,Total Complaints
0,October,"160,136","267,121","7,125","434,382"
1,November,"149,543","266,448","6,793","422,784"
2,December,"126,110","229,747","6,002","361,859"
3,January,"154,608","277,461","7,186","439,255"
4,February,"166,369","307,544","7,620","481,533"
5,March,"173,516","326,772","8,460","508,748"
6,April,"165,212","329,698","8,573","503,483"
7,May,"172,867","393,343","8,876","575,086"
8,June,"155,039","354,663","8,340","518,042"
9,July,"153,503","345,222","8,545","507,270"
